# Context-FID Score Presentation
## Necessary packages and functions call

- Context-FID score: A useful metric measures how well the the synthetic time series windows ”fit” into the local context of the time series

In [1]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

## Data Loading  Morning Peak

Load original dataset and preprocess the loaded data.

In [2]:
# iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# # ori_data = np.load('../OUTPUT/{dataset_name}/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../toy_exp/ddpm_fake_sines.npy')


iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# ori_data = np.load('../OUTPUT/test_ep/samples/etth_norm_truth_24_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../OUTPUT/test_ep/ddpm_fake_test_ep_milestone_10.npy')

ori_data = np.load('../OUTPUT/etth_mpep/samples/morning_peak_etth_norm_truth_24_train.npy')  # Uncomment the line if dataset other than Sine is used.
fake_data = np.load('../OUTPUT/etth_mpep/ddpm_fake_morning_peak_etth_milestone_500.npy')

print('ori shape is: ', ori_data.shape)
print('fake shape is: ', fake_data.shape)
b,t,n = ori_data.shape


ori_data = ori_data.transpose(2, 0, 1).reshape(b * n, t, 1)

# fake_data = fake_data[:ori_data.shape[0]*ori_data.shape[2]]
# fake_data = fake_data.reshape(n, b, t).transpose(1, 2, 0)

print('ori shape is: ', ori_data.shape)
print('fake shape is: ', fake_data.shape)

ori shape is:  (2881, 24, 7)
fake shape is:  (20700, 24, 1)
ori shape is:  (20167, 24, 1)
fake shape is:  (20700, 24, 1)








## Context-FID Score

- The Frechet Inception distance-like score is based on unsupervised time series embeddings. It is able to score the fit of the fixed length synthetic samples into their context of (often much longer) true time series.

- The lowest scoring models correspond to the best performing models in downstream tasks

In [3]:
for j in range(3):

    context_fid_score = []

    for i in range(iterations):
        context_fid = Context_FID(ori_data[:], fake_data[:ori_data.shape[0]])
        context_fid_score.append(context_fid)
        print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')

    display_scores(context_fid_score)

# Seed 12345 :  mp  10mile  0.292   0.309
# seed 12345 :  ep  10mile  0.151


# mp 24  mix etth mpep input,   




Iter 0:  context-fid = 0.10202260536765614 

Iter 1:  context-fid = 0.09069351036866274 

Iter 2:  context-fid = 0.0998399036499504 

Iter 3:  context-fid = 0.11043615429809825 

Iter 4:  context-fid = 0.08376089242374185 

Final Score:  0.09735061322162189 ± 0.01284766773866546
Iter 0:  context-fid = 0.08768118198323899 

Iter 1:  context-fid = 0.09008245031905535 

Iter 2:  context-fid = 0.09328211804942788 

Iter 3:  context-fid = 0.12893061594702643 

Iter 4:  context-fid = 0.12215831471610569 

Final Score:  0.10442693620297086 ± 0.024246042205174866
Iter 0:  context-fid = 0.1009398038946919 

Iter 1:  context-fid = 0.13728129345551981 

Iter 2:  context-fid = 0.09482702397469475 

Iter 3:  context-fid = 0.10773269500828958 

Iter 4:  context-fid = 0.09805075323734214 

Final Score:  0.10776631391410763 ± 0.02132224638743598


## Correlational Score

- The metric uses the absolute error of the auto-correlation estimator by real data and synthetic data as the metric to assess the temporal dependency.

- For d > 1, it uses the l1-norm of the difference between cross correlation matrices.

In [4]:
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

In [5]:
x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

# single input    mp 10miles    0.087 + 0.02
# single input    ep 10miles    0.104 + 0.03

Iter 0:  cross-correlation = 1.432187701766452e-15 

Iter 1:  cross-correlation = 1.4099832412739487e-15 

Iter 2:  cross-correlation = 1.2323475573339237e-15 

Iter 3:  cross-correlation = 2.930988785010413e-15 

Iter 4:  cross-correlation = 7.216449660063518e-16 

Final Score:  1.5454304502782177e-15 ± 1.0253290331335782e-15


## Data Loading Evening Peak

Load original dataset and preprocess the loaded data.

In [6]:
# iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# # ori_data = np.load('../OUTPUT/{dataset_name}/samples/{dataset_name}_norm_truth_{seq_length}_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../toy_exp/ddpm_fake_sines.npy')


iterations = 5
# ori_data = np.load('../toy_exp/samples/sine_ground_truth_24_train.npy')
# ori_data = np.load('../OUTPUT/test_ep/samples/etth_norm_truth_24_train.npy')  # Uncomment the line if dataset other than Sine is used.
# fake_data = np.load('../OUTPUT/test_ep/ddpm_fake_test_ep_milestone_10.npy')

ori_data = np.load('../OUTPUT/etth_mpep/samples/evening_peak_etth_norm_truth_24_train.npy')  # Uncomment the line if dataset other than Sine is used.
fake_data = np.load('../OUTPUT/etth_mpep/ddpm_fake_evening_peak_etth_milestone_500.npy')

print('ori shape is: ', ori_data.shape)
print('fake shape is: ', fake_data.shape)
b,t,n = ori_data.shape


ori_data = ori_data.transpose(2, 0, 1).reshape(b * n, t, 1)

# fake_data = fake_data[:ori_data.shape[0]*ori_data.shape[2]]
# fake_data = fake_data.reshape(n, b, t).transpose(1, 2, 0)

print('ori shape is: ', ori_data.shape)
print('fake shape is: ', fake_data.shape)

ori shape is:  (2880, 24, 7)
fake shape is:  (20700, 24, 1)
ori shape is:  (20160, 24, 1)
fake shape is:  (20700, 24, 1)








## Context-FID Score

- The Frechet Inception distance-like score is based on unsupervised time series embeddings. It is able to score the fit of the fixed length synthetic samples into their context of (often much longer) true time series.

- The lowest scoring models correspond to the best performing models in downstream tasks

In [7]:
for j in range(3):

    context_fid_score = []

    for i in range(iterations):
        context_fid = Context_FID(ori_data[:], fake_data[:ori_data.shape[0]])
        context_fid_score.append(context_fid)
        print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')

    display_scores(context_fid_score)

# Seed 12345 :  mp  10mile  0.292   0.309
# seed 12345 :  ep  10mile  0.151


# mp 24 是0.047， 96 就0.2了应该，但是gpt的24是0.02

#  ep 24 是  0.0123


Iter 0:  context-fid = 0.04408167500634456 

Iter 1:  context-fid = 0.028713816672307654 

Iter 2:  context-fid = 0.043102472912550274 

Iter 3:  context-fid = 0.04024633937982867 

Iter 4:  context-fid = 0.043523238015058974 

Final Score:  0.03993350839721803 ± 0.008001634029997184
Iter 0:  context-fid = 0.0339054057788801 

Iter 1:  context-fid = 0.027146468529383953 

Iter 2:  context-fid = 0.03908637152651748 

Iter 3:  context-fid = 0.03896425899266845 

Iter 4:  context-fid = 0.04202412330468053 

Final Score:  0.0362253256264261 ± 0.0072698623462340165
Iter 0:  context-fid = 0.05322801443432249 

Iter 1:  context-fid = 0.03567782327446088 

Iter 2:  context-fid = 0.04932337001794816 

Iter 3:  context-fid = 0.04830015896868775 

Iter 4:  context-fid = 0.02307292187508076 

Final Score:  0.04192045771410001 ± 0.015428201434468249


## Correlational Score

- The metric uses the absolute error of the auto-correlation estimator by real data and synthetic data as the metric to assess the temporal dependency.

- For d > 1, it uses the l1-norm of the difference between cross correlation matrices.

In [8]:
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

In [9]:
x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

# single input    mp 10miles    0.087 + 0.02
# single input    ep 10miles    0.104 + 0.03

Iter 0:  cross-correlation = 1.5321077739827161e-15 

Iter 1:  cross-correlation = 1.4988010832439613e-15 

Iter 2:  cross-correlation = 2.253752739989068e-15 

Iter 3:  cross-correlation = 3.519406988061746e-15 

Iter 4:  cross-correlation = 5.329070518200751e-16 

Final Score:  1.8673951274195133e-15 ± 1.37532897420109e-15
